# HW2_CSCI544
## NAME : YAMINI HARIPRIYA
## USC ID : 9195653004

# 1. Dataset Generation

Some of the tools that are added to the code at the start of the process are NumPy, Pandas, Regular Expressions, BeautifulSoup, scikit-learn, Gensim, tqdm, NLTK, and Torch. After that, Pandas is used to read the information from a TSV file. In more detail, it chooses the "review_body" and "star_rating" fields and sets the delimiter and encoding at the same time. The DataFrame is made even by picking 50,000 reviews for each rating from 1 to 5, after any NaN values have been removed and the "star_rating" column has been changed to a number format. This makes a DataFrame that is balanced, which is called "balanced_data."
** **
A preprocessing function named 'preprocess_text' is defined to lowercase text, remove HTML tags, URLs, non-alphabetic characters, and stopwords, and lemmatize the remaining tokens. This function is applied to the 'Review' column of the 'balanced_data' DataFrame, thereby cleaning the text data.
** **
Sentiment labels ('sentiment_class') are created based on star ratings, where ratings greater than 3 are labeled as positive (1), ratings less than 3 are labeled as negative (2), and ratings equal to 3 are labeled as neutral (0). The dataset is then split into features ('X_train' and 'X_test') and labels ('y_train' and 'y_test') using the 'train_test_split' function from scikit-learn, with a test size of 20% and a random state of 42

In [1]:
import sys

# Print Python version
print("Python Version:", sys.version)


In [2]:
# Print Python dependencies
print("\nDependencies:")
for module in sys.modules:
    try:
        module_version = sys.modules[module].__version__
        print(module, ":", module_version)
    except AttributeError:
        pass

### Import Statements

In [3]:
import numpy as np  
import pandas as pd  
import re           
from bs4 import BeautifulSoup 
from sklearn.model_selection import train_test_split  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import gensim.downloader as api
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

In [4]:
import nltk         
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') 
nltk.download('stopwords')  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yamini_Kanuparthi\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yamini_Kanuparthi\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Loading dataset with only necessary columns
path = r"C:\Users\Yamini_Kanuparthi\Downloads\amazon_reviews_us_Office_Products_v1_00.tsv"
Sentiment_df = pd.read_csv(path,usecols=['review_body', 'star_rating'], delimiter='\t', encoding='utf-8', error_bad_lines=False)

C:\Users\Yamini_Kanuparthi\AppData\Local\Temp\ipykernel_12756\3307972338.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Sentiment_df = pd.read_csv(path,usecols=['review_body', 'star_rating'], delimiter='\t', encoding='utf-8', error_bad_lines=False)
C:\Users\Yamini_Kanuparthi\AppData\Local\Temp\ipykernel_12756\3307972338.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  Sentiment_df = pd.read_csv(path,usecols=['review_body', 'star_rating'], delimiter='\t', encoding='utf-8', error_bad_lines=False)


In [6]:
# Preprocessing: Cleaning and balancing dataset
Sentiment_df.dropna(inplace=True)
Sentiment_df['star_rating'] = pd.to_numeric(Sentiment_df['star_rating'], errors='coerce')
Sentiment_df.rename(columns={'review_body': 'Review', 'star_rating': 'Rating'}, inplace=True)

# Balancing dataset
balanced_data = pd.DataFrame()
for rating in range(1, 6):  # Assuming ratings are from 1 to 5
    subset = Sentiment_df[Sentiment_df['Rating'] == rating]
    balanced_subset = subset.sample(n=50000, random_state=42)
    balanced_data = pd.concat([balanced_data, balanced_subset], ignore_index=True)



In [7]:
balanced_data.shape

(250000, 2)

In [8]:
balanced_data.head()

,Rating,Review
0,1.0,This is one of the worst purchases I have ever...
1,1.0,Applied correctly to winter coat. Did not last...
2,1.0,This product broke in a very short time. It a...
3,1.0,Two of the four ink cartridges were empty. You...
4,1.0,"My SO and I have owned two of these, and they ..."


In [9]:
!pip install contractions

In [10]:
import contractions

In [11]:
# Combining text preprocessing steps into a single operation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'http\S+', '', text)
    text = contractions.fix(text)
    text = re.sub(r"[^a-zA-Z\s]", '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

balanced_data['Review'] = balanced_data['Review'].apply(preprocess_text)

# Creating labels and splitting dataset
balanced_data['sentiment_class'] = balanced_data['Rating'].apply(lambda x: 1 if x > 3 else (2 if x < 3 else 0))
X_train, X_test, y_train, y_test = train_test_split(balanced_data['Review'], balanced_data['sentiment_class'], test_size=0.2, random_state=42)


C:\Users\Yamini_Kanuparthi\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Yamini_Kanuparthi\anaconda3\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


# 2. Word Embedding  

## 2 a) Here I have used Gensim Library to load Word2Vec model. 


The first part of the code installs the Gensim library for inserting words. Using Gensim's API, it loads a Word2Vec model that has already been trained: "word2vec-google-news-300." Then, vector processes are used to quickly find words that are similar. For example, the vectors for "king," "man," and "woman" are changed to create a new vector. Using the pre-trained model, the code writes words that are similar and words that have semantic similarities.


After the text input has been cleaned up, a custom Word2Vec model is trained on it. The code creates a function that finds similarities between vectors and then uses that function to find the similarities between the changed vector ('queen' - 'king' -'man' + 'woman') and 'queen'. The custom model also checks to see if certain words have semantic connections.


Furthermore, binary labels are prepared for sentiment analysis (positive for ratings > 3, negative for ratings < 3). The dataset is split into training and testing sets for binary sentiment classification, and binary labels are created accordingly.

## Pretrained Word2Vec Model

In [12]:
!pip install gensim

In [13]:
# Load the pre-trained Word2Vec model
wv = api.load('word2vec-google-news-300')

# Perform vector operations and find similar words efficiently
king_vec = wv['king']
man_vec = wv['man']
woman_vec = wv['woman']
new_vector = king_vec - man_vec + woman_vec
similar_words = wv.similar_by_vector(new_vector)
print(similar_words)


[('king', 0.8449392318725586), ('queen', 0.7300517559051514), ('monarch', 0.645466148853302), ('princess', 0.6156251430511475), ('crown_prince', 0.5818676352500916), ('prince', 0.5777117609977722), ('kings', 0.5613664388656616), ('sultan', 0.5376776456832886), ('Queen_Consort', 0.5344247221946716), ('queens', 0.5289887189865112)]


In [14]:
# Check semantic similarities using the pre-trained model
similarities = wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(f"Semantic similarities (woman + king - man): {similarities}")



Semantic similarities (woman + king - man): [('queen', 0.7118192911148071)]


In [15]:
# Check similarity for specific words
similarity = wv.similarity('excellent', 'outstanding')
print(f"Similarity between 'excellent' and 'outstanding': {similarity}")

Similarity between 'excellent' and 'outstanding': 0.556748628616333


In [16]:
# Check similarity for specific words
similarity = wv.similarity('king', 'queen')
print(f"Similarity between 'king' and 'queen': {similarity}")

Similarity between 'king' and 'queen': 0.6510956883430481


## Custom Word2Vec Model

In [17]:
sentences = [word_tokenize(review.lower()) for review in balanced_data['Review']]


In [18]:
# Train the Word2Vec model
model_cust = Word2Vec(sentences, vector_size=300, window=11, min_count=10, workers=4)

def similarityFunc(word1, word2):
  return np.dot(word1, word2)/(np.linalg.norm(word1)* np.linalg.norm(word2))

# Calculate the vector for "king" - "man" + "woman"
if all(word in model_cust.wv.key_to_index for word in ['king', 'man', 'woman', 'queen']):
    result_vector = model_cust.wv['king'] - model_cust.wv['man'] + model_cust.wv['woman']
    queen_vector = model_cust.wv['queen']

    # Calculate the similarity score
    similar_score_custom = similarityFunc(result_vector, queen_vector)
    print("The Similarity between the (King- Man + Woman) and (Queen) with custom model is", similar_score_custom)

# Check semantic similarities using the custom model
similarity = model_cust.wv.similarity("excellent", "outstanding")
print("Similarity between EXCELLENT and OUTSTANDING is", similarity)

similarity = model_cust.wv.similarity("king", "queen")
print("Similarity between KING and QUEEN is", similarity)


The Similarity between the (King- Man + Woman) and (Queen) with custom model is 0.2535056
Similarity between EXCELLENT and OUTSTANDING is 0.76753366
Similarity between KING and QUEEN is 0.39628977


** **
** **
** **
** **
** **
** **

### What do you conclude from comparing vectors generated by yourself and the pretrained model? Which of the Word2Vec models seems to encode semantic similarities between words better?

Comparing the vectors generated by the pretrained model and the custom-trained model, we observe some differences in semantic similarities between words. 

In the pretrained model:
- The vector operations 'King Man+Woman=Queen' yield similar words like 'queen', 'monarch', 'princess', indicating a strong semantic relationship.
- The semantic similarity between 'excellent' and 'outstanding' is moderate at 0.5567.
- The similarity between 'king' and 'queen' is also moderate at 0.65109.
** **
In the custom-trained model:
- The vector operations for 'King Man+Woman=Queen' result in a similarity score of 0.2648 between the computed vector and 'queen', which is lower compared to the pretrained model where it is 0.7118.
- The semantic similarity between 'excellent' and 'outstanding' is higher at 0.7984.
- The similarity between 'king' and 'queen' is lower compared to the pretrained model, indicating that the custom-trained model may not capture this relationship as effectively.
** **
When comparing the similarity between (King - Man + Woman) and (Queen), we can see that the custom trained model is not as good as the pre-trained one, since the pre-trained one has a semantic similarity of 71% while the custom model has a similarity of 21%. However, when we look at the semantic similarity between EXCELLENT and OUTSTANDING, the semantic similarity of the custom trained model is great. The issue could be because Word2Vec is trained on a large corpus, thus there may be many terms closer to EXCELLENT and OUTSTANDING, lowering the relative similarity. 

From this comparison, it appears that the pretrained Word2Vec model tends to encode semantic similarities between words better, as evidenced by the higher similarity scores for vector operations and word pairs. However, the custom-trained model may still provide meaningful embeddings tailored to the specific dataset. Overall, the pretrained model demonstrates a stronger ability to capture semantic relationships present in a broader corpus of text data.






In [14]:
binary_data = balanced_data[balanced_data['Rating'] != 3]

# Prepare binary labels: 1 for positive (ratings > 3) and 0 for negative (ratings < 3)
binary_data['binary_sentiment'] = binary_data['Rating'].apply(lambda x: 1 if x > 3 else 0)

C:\Users\Yamini_Kanuparthi\AppData\Local\Temp\ipykernel_13292\1444351052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_data['binary_sentiment'] = binary_data['Rating'].apply(lambda x: 1 if x > 3 else 0)


In [15]:
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(binary_data['Review'], binary_data['binary_sentiment'], test_size=0.2, random_state=42)


In [16]:
X_train_binary.shape

(160000,)

# 3 Simple models

Thos part aims to generate Word2Vec features for the binary sentiment classification task using pretrained and custom-trained Word2Vec models. It then trains Perceptron and SVM models using these features and compares their accuracy with models trained using TF-IDF features. Below is a summary of the code:

1. **Word2Vec Feature Extraction:**
   - Word embeddings are extracted for training and testing data using both the pretrained ('word2vec-google-news-300') and custom-trained Word2Vec models.
   - For each review, the average Word2Vec vectors are computed by summing up the word vectors and dividing by the number of words.
   
  ** ** 
** **
  
** **
  
   
2. **Model Training and Evaluation:**
   - Perceptron and SVM models are trained using Word2Vec features extracted from both pretrained and custom-trained models.
   - Accuracy scores are computed for each model using the test data, and the results are printed.
   
   ** ** 
   
3. **TF-IDF Feature Extraction and Model Training:**
   - TF-IDF features are generated using the TfidfVectorizer from scikit-learn.
   - Perceptron and SVM models are trained using TF-IDF features, and their accuracy scores are computed and printed.



In [17]:
def compute_document_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model


In [18]:
binary_word_vectors_train = []

# Extract Word2Vec features for training data
for e in tqdm(X_train_binary):
    binary_word_vectors_train.append(compute_document_embedding(e))


binary_word_vectors_train = np.array(binary_word_vectors_train)



100%|██████████| 160000/160000 [02:24<00:00, 1104.58it/s]


In [19]:
binary_word_vectors_test = []

# Extract Word2Vec features for testing data
for e in tqdm(X_test_binary):
    binary_word_vectors_test.append(compute_document_embedding(e))
    
binary_word_vectors_test = np.array(binary_word_vectors_test)


100%|██████████| 40000/40000 [00:37<00:00, 1062.06it/s]


In [20]:
def compute_document_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return model_cust.wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model



In [21]:
bin_cust_w2v_train = []


# Extract Word2Vec features for training data
for e in tqdm(X_train_binary):
    bin_cust_w2v_train.append(compute_document_embedding(e))
    
bin_cust_w2v_train = np.array(bin_cust_w2v_train)




100%|██████████| 160000/160000 [02:12<00:00, 1208.08it/s]


In [22]:
bin_cust_w2v_test = []
    
# Extract Word2Vec features for testing data
for e in tqdm(X_test_binary):
    bin_cust_w2v_test.append(compute_document_embedding(e))
    
bin_cust_w2v_test = np.array(bin_cust_w2v_test)


100%|██████████| 40000/40000 [00:32<00:00, 1229.40it/s]


In [23]:
bin_cust_w2v_train.shape, bin_cust_w2v_test.shape

((160000, 300), (40000, 300))

## Perceptron and SVM for Pretrained Word2Vec - Binary Classification

In [25]:
perceptron_pretrained = Perceptron(max_iter=1000)
perceptron_pretrained.fit(binary_word_vectors_train, y_train_binary)

predictions = perceptron_pretrained.predict(binary_word_vectors_test)
accuracy = accuracy_score(y_test_binary, predictions)
print("Perceptron (Pretrained Word2Vec) Accuracy:", accuracy)

Perceptron (Pretrained Word2Vec) Accuracy: 0.5425


In [26]:
svm_pretrained = LinearSVC()
svm_pretrained.fit(binary_word_vectors_train, y_train_binary)

predictions = svm_pretrained.predict(binary_word_vectors_test)
accuracy = accuracy_score(y_test_binary, predictions)
print("LinearSVC (Pretrained Word2Vec) Accuracy:", accuracy)

LinearSVC (Pretrained Word2Vec) Accuracy: 0.61595


## Perceptron and SVM for Custom Word2Vec - Binary Clasification

In [27]:
perceptron_custom = Perceptron()
perceptron_custom.fit(bin_cust_w2v_train, y_train_binary)

predictions = perceptron_custom.predict(bin_cust_w2v_test)
accuracy = accuracy_score(y_test_binary, predictions)
print("Perceptron (Custom Word2Vec) Accuracy:", accuracy)

Perceptron (Custom Word2Vec) Accuracy: 0.525875


In [29]:
svm_custom = LinearSVC()
svm_custom.fit(bin_cust_w2v_train, y_train_binary)

predictions = svm_custom.predict(bin_cust_w2v_test)
accuracy = accuracy_score(y_test_binary, predictions)
print("LinearSVC (Custom Word2Vec) Accuracy:", accuracy)


LinearSVC (Custom Word2Vec) Accuracy: 0.61815


## Perceptron and SVM Training -  TF- IDF features

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Assuming 'Review' is your preprocessed text column in 'binary_data'
text_data = binary_data['Review']

# Fit and transform the text data to get TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(text_data)


# Use the 'binary_sentiment' column as labels
y = binary_data['binary_sentiment']

# Split the TF-IDF features and labels into training and testing sets
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [31]:
perceptron_tfidf = Perceptron()
perceptron_tfidf.fit(X_train_tfidf, y_train_tfidf)

predictions = perceptron_tfidf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test_tfidf, predictions)
print("Perceptron (TF-IDF) Accuracy:", accuracy)

Perceptron (TF-IDF) Accuracy: 0.81385


In [32]:
svm_tfidf = LinearSVC()
svm_tfidf.fit(X_train_tfidf, y_train_tfidf)

predictions = svm_tfidf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test_tfidf, predictions)
print("LinearSVC (TF-IDF) Accuracy:", accuracy)

LinearSVC (TF-IDF) Accuracy: 0.863825


### What do you conclude from comparing performances for the models trained using the three different feature types (TF-IDF, pretrained Word2Vec, your trained Word2Vec)?


From the obtained accuracy scores, we can draw the following conclusions regarding the performances of models trained using different feature types:

1. **TF-IDF Features:** The Perceptron and LinearSVC models trained using TF-IDF features achieve the highest accuracy scores among all models. This indicates that TF-IDF features are highly effective in capturing the discriminative information necessary for sentiment classification. The Perceptron model achieves an accuracy of approximately 81.39%, while the LinearSVC model achieves an accuracy of approximately 86.38%. TF-IDF is a straightforward and efficient approach that often yields satisfactory results for smaller datasets. It can effectively identify the most pertinent words and phrases that relate to the sentiment of a given document, making it suitable for sentiment analysis tasks.
** **
2. **Pretrained Word2Vec Features:** Models trained using Word2Vec features extracted from the pretrained model achieve moderate accuracy scores. The Single Layer Perceptron using pretrained Word2Vec features achieves an accuracy of approximately 54.25%, while the SVM model achieves an accuracy of approximately 61.60%. Pretrained Word2Vec embeddings are typically beneficial when processing large quantities of textual data, as they are capable of capturing semantic relationships between words and phrases. They can be particularly effective for sentiment analysis tasks involving slang, idiomatic expressions, and other colloquial language that may pose challenges for traditional bag-of-words methods like TF-IDF.
** **
3. **Custom Word2Vec Features:** Models trained using Word2Vec features from the custom-trained model exhibit slightly improved accuracy compared to models using pretrained Word2Vec features. The Single Layer Perceptron achieves an accuracy of approximately 52.0%, while the SVM model achieves an accuracy of approximately 61.83%. However, these accuracy scores are still lower than those obtained using TF-IDF features, indicating that the custom Word2Vec features may not be as effective as TF-IDF features for sentiment classification on this dataset. Nonetheless, it is reasonable to speculate that for more extensive and intricate datasets, Word2Vec may have the upper hand.

In summary, TF-IDF features outperform both pretrained and custom Word2Vec features for sentiment classification on the given dataset. While Word2Vec features capture semantic information, TF-IDF features seem to better capture the discriminative information necessary for sentiment analysis in this context. However, the choice between TF-IDF and Word2Vec depends on the characteristics of the dataset and the specific requirements of the sentiment analysis task.

** **
** **

** **



# 4. Feedforward Neural Networks 


### 4a) Using Average Word2Vec Vectors:
1. **Binary Classification:**
   - Concatenated Word2Vec vectors were used as input features for the binary classification model.
   - A binary MLP model with two hidden layers (50 and 10 nodes) and softmax activation in the output layer was defined.
   - The model was trained using cross-entropy loss and Adam optimizer.
   - Accuracy was evaluated on the testing split.
** **

2. **Ternary Classification:**
   - Similar steps were followed as for binary classification, but with three output classes.
   - The model was trained and evaluated using the same approach as for binary classification.

### 4b) Concatenating Word2Vec Vectors:
1. **Binary Classification:**
   - The first 10 Word2Vec vectors for each review were concatenated to form the input features.
   - The binary MLP model was redefined, trained, and evaluated using the concatenated features.
** **
** **
** **

2. **Ternary Classification:**
   - Similar steps were followed as for binary classification, but with three output classes.
   - The model was trained and evaluated using the concatenated features.



In [33]:
# Convert labels to binary format (0 for class 1, 1 for class 2)

y_np_train = np.array(y_train_binary)
y_np_test = np.array(y_test_binary)

y_train_binary = np.where(y_np_train == 1, 0, 1)
y_test_binary = np.where(y_np_test == 1, 0, 1)



In [34]:
# Define the MLP Model for binary classification
class BinaryMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(BinaryMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

In [35]:
# Training Loop with tqdm progress bar
def train_with_progress_bar(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                pbar.set_postfix({'loss': f'{running_loss/len(train_loader):.3f}'})
                pbar.update()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.3f}")


# Evaluation
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

## Multi Layer Perceptron with Binary Classification - Pretrained Features

In [41]:
# Convert data to PyTorch tensors
train_dataset_pre_binary = TensorDataset(torch.tensor(binary_word_vectors_train, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long))
test_dataset_pre_binary = TensorDataset(torch.tensor(binary_word_vectors_test, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long))

# Create DataLoader
train_loader_pre_binary = DataLoader(train_dataset_pre_binary, batch_size=32, shuffle=True)
test_loader_pre_binary = DataLoader(test_dataset_pre_binary, batch_size=32)



In [42]:
# Initialize the model
input_size = binary_word_vectors_train.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification (classes 1 and 2)
binary_model_pre_binary = BinaryMLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(binary_model_pre_binary.parameters(), lr=0.001)

# Train the binary classification model
num_epochs = 10
train_with_progress_bar(binary_model_pre_binary, train_loader_pre_binary, criterion, optimizer, num_epochs)

# Evaluate the binary classification model
evaluate(binary_model_pre_binary, test_loader_pre_binary)

Epoch 1/10: 100%|██████████| 5000/5000 [00:20<00:00, 238.82batch/s, loss=0.668]


Epoch 1/10, Loss: 3339.553


Epoch 2/10: 100%|██████████| 5000/5000 [00:23<00:00, 215.30batch/s, loss=0.653]


Epoch 2/10, Loss: 3264.732


Epoch 3/10: 100%|██████████| 5000/5000 [00:21<00:00, 236.26batch/s, loss=0.647]


...

Epoch 8/10, Loss: 3197.777


Epoch 9/10: 100%|██████████| 5000/5000 [00:21<00:00, 227.46batch/s, loss=0.639]


Epoch 9/10, Loss: 3194.755


Epoch 10/10: 100%|██████████| 5000/5000 [00:22<00:00, 219.89batch/s, loss=0.638]


Epoch 10/10, Loss: 3191.535
Accuracy: 0.639625


## Multi Layer Perceptron with Binary Classification - Custom Word2Vec Features

In [38]:
# Convert data to PyTorch tensors
train_dataset_cust_binary = TensorDataset(torch.tensor(bin_cust_w2v_train, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long))
test_dataset_cust_binary = TensorDataset(torch.tensor(bin_cust_w2v_test, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long))

# Create DataLoader
train_loader_cust_binary = DataLoader(train_dataset_cust_binary, batch_size=32, shuffle=True)
test_loader_cust_binary = DataLoader(test_dataset_cust_binary, batch_size=32)



In [40]:
# Initialize the model
input_size = bin_cust_w2v_train.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification (classes 1 and 2)
binary_model_cust_binary = BinaryMLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(binary_model_cust_binary.parameters(), lr=0.001)

# Train the binary classification model
num_epochs = 10
train_with_progress_bar(binary_model_cust_binary, train_loader_cust_binary, criterion, optimizer, num_epochs)

# Evaluate the binary classification model
evaluate(binary_model_cust_binary, test_loader_cust_binary)

Epoch 1/10: 100%|██████████| 5000/5000 [00:17<00:00, 288.61batch/s, loss=0.664]


Epoch 1/10, Loss: 3318.768


Epoch 2/10: 100%|██████████| 5000/5000 [00:19<00:00, 253.90batch/s, loss=0.651]


Epoch 2/10, Loss: 3253.512


Epoch 3/10: 100%|██████████| 5000/5000 [00:20<00:00, 245.89batch/s, loss=0.645]


...

Epoch 8/10, Loss: 3185.489


Epoch 9/10: 100%|██████████| 5000/5000 [00:20<00:00, 240.49batch/s, loss=0.636]


Epoch 9/10, Loss: 3181.307


Epoch 10/10: 100%|██████████| 5000/5000 [00:22<00:00, 223.26batch/s, loss=0.636]


Epoch 10/10, Loss: 3178.313
Accuracy: 0.638125


## Multi Layer Perceptron with Ternary Classification

In [43]:
def compute_document_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return model_cust.wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model



In [44]:
ter_cust_w2v_train = []

# Extract Word2Vec features for training data
for e in tqdm(X_train):
    ter_cust_w2v_train.append(compute_document_embedding(e))

ter_cust_w2v_train = np.array(ter_cust_w2v_train)




100%|██████████| 200000/200000 [02:47<00:00, 1193.70it/s]


In [45]:
ter_cust_w2v_test = []

# Extract Word2Vec features for testing data
for e in tqdm(X_test):
    ter_cust_w2v_test.append(compute_document_embedding(e))
    
ter_cust_w2v_test = np.array(ter_cust_w2v_test)

100%|██████████| 50000/50000 [00:42<00:00, 1176.30it/s]


In [46]:
ter_cust_w2v_train.shape, ter_cust_w2v_test.shape

((200000, 300), (50000, 300))

In [47]:
def compute_document_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model
    


In [48]:
ternary_word_vectors_train = []

# Extract Word2Vec features for training data
for e in tqdm(X_train):
    ternary_word_vectors_train.append(compute_document_embedding(e))
    
ternary_word_vectors_train = np.array(ternary_word_vectors_train)

100%|██████████| 200000/200000 [02:58<00:00, 1117.76it/s]


In [49]:
ternary_word_vectors_test = []


# Extract Word2Vec features for testing data
for e in tqdm(X_test):
    ternary_word_vectors_test.append(compute_document_embedding(e))
    
ternary_word_vectors_test = np.array(ternary_word_vectors_test)

100%|██████████| 50000/50000 [00:44<00:00, 1120.76it/s]


In [50]:
ternary_word_vectors_train.shape, ternary_word_vectors_test.shape

((200000, 300), (50000, 300))

In [51]:
# Convert labels to NumPy arrays
# Convert labels to integers ranging from 0 to 2

y_np_train = np.array(y_train)
y_np_test = np.array(y_test)

y_train_encoded = [label for label in y_np_train]  
y_test_encoded = [label for label in y_np_test]  




In [52]:
# Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)



In [53]:
# Training Loop
# Training Loop with tqdm progress bar
def train_with_progress_bar(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                pbar.set_postfix({'loss': f'{running_loss/len(train_loader):.3f}'})
                pbar.update()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


# Evaluation
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

## Multi Layer Perceptron with Ternary Classification - Pretrained Features

In [54]:
# Convert data to PyTorch tensors
train_dataset_pre_ternary = TensorDataset(torch.tensor(ternary_word_vectors_train, dtype=torch.float32), torch.tensor(y_train_encoded, dtype=torch.long))
test_dataset_pre_ternary = TensorDataset(torch.tensor(ternary_word_vectors_test, dtype=torch.float32), torch.tensor(y_test_encoded, dtype=torch.long))

# Create DataLoader
train_loader_pre_ternary = DataLoader(train_dataset_pre_ternary, batch_size=32, shuffle=True)
test_loader_pre_ternary = DataLoader(test_dataset_pre_ternary, batch_size=32)



In [55]:
# Initialize the model
input_size = ternary_word_vectors_train.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
ternary_model_pre = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ternary_model_pre.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(ternary_model_pre, train_loader_pre_ternary, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(ternary_model_pre, test_loader_pre_ternary)

Epoch 1/10: 100%|██████████| 6250/6250 [00:27<00:00, 228.68batch/s, loss=1.041]


Epoch 1/10, Loss: 6504.971320569515


Epoch 2/10: 100%|██████████| 6250/6250 [00:27<00:00, 223.54batch/s, loss=1.027]


Epoch 2/10, Loss: 6416.973695039749


Epoch 3/10: 100%|██████████| 6250/6250 [00:30<00:00, 208.22batch/s, loss=1.022]


...

Epoch 8/10, Loss: 6318.43892544508


Epoch 9/10: 100%|██████████| 6250/6250 [00:30<00:00, 206.89batch/s, loss=1.010]


Epoch 9/10, Loss: 6312.607225835323


Epoch 10/10: 100%|██████████| 6250/6250 [00:30<00:00, 207.98batch/s, loss=1.009]


Epoch 10/10, Loss: 6307.618082404137
Accuracy: 0.51178


## Multi Layer Perceptron with Ternary Classification - Custom Word2Vec Features

In [56]:
# Convert data to PyTorch tensors
train_dataset_cust_ternary = TensorDataset(torch.tensor(ter_cust_w2v_train, dtype=torch.float32), torch.tensor(y_train_encoded, dtype=torch.long))
test_dataset_cust_ternary = TensorDataset(torch.tensor(ter_cust_w2v_test, dtype=torch.float32), torch.tensor(y_test_encoded, dtype=torch.long))

# Create DataLoader
train_loader_cust_ternary = DataLoader(train_dataset_cust_ternary, batch_size=32, shuffle=True)
test_loader_cust_ternary = DataLoader(test_dataset_cust_ternary, batch_size=32)



In [57]:
# Initialize the model
input_size = ter_cust_w2v_train.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
Ternary_model_cust = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Ternary_model_cust.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(Ternary_model_cust, train_loader_cust_ternary, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(Ternary_model_cust, test_loader_cust_ternary)

Epoch 1/10: 100%|██████████| 6250/6250 [00:30<00:00, 202.70batch/s, loss=1.041]


Epoch 1/10, Loss: 6504.433157503605


Epoch 2/10: 100%|██████████| 6250/6250 [00:32<00:00, 194.03batch/s, loss=1.022]


Epoch 2/10, Loss: 6386.571774423122


Epoch 3/10: 100%|██████████| 6250/6250 [00:31<00:00, 198.62batch/s, loss=1.018]


...

Epoch 8/10, Loss: 6316.446863055229


Epoch 9/10: 100%|██████████| 6250/6250 [00:31<00:00, 196.15batch/s, loss=1.010]


Epoch 9/10, Loss: 6313.760311603546


Epoch 10/10: 100%|██████████| 6250/6250 [00:32<00:00, 194.12batch/s, loss=1.010]


Epoch 10/10, Loss: 6311.003338396549
Accuracy: 0.50586


## First 10 Word2Vec vectors on Ternary Classification - Custom Word2Vec Model

In [58]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_conc_custTer = []
X_test_con_custTer = []

for review in ter_cust_w2v_train:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_conc_custTer.append(concatenated_vector)

for review in ter_cust_w2v_test:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_con_custTer.append(concatenated_vector)



In [59]:
# Convert the concatenated features into PyTorch tensors
X_train_concat_tensor_cust = torch.tensor(X_train_conc_custTer, dtype=torch.float32)
X_test_concat_tensor_cust = torch.tensor(X_test_con_custTer, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor_cust = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor_cust = torch.tensor(y_test_encoded, dtype=torch.long)

# Create DataLoader
train_dataset_con_custTer = TensorDataset(X_train_concat_tensor_cust, y_train_tensor_cust)
test_dataset_con_custTer = TensorDataset(X_test_concat_tensor_cust, y_test_tensor_cust)

train_loader_con_custTer = DataLoader(train_dataset_con_custTer, batch_size=32, shuffle=True)
test_loader_con_custTer = DataLoader(test_dataset_con_custTer, batch_size=32)



C:\Users\Yamini_Kanuparthi\AppData\Local\Temp\ipykernel_13292\687041903.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  X_train_concat_tensor_cust = torch.tensor(X_train_conc_custTer, dtype=torch.float32)


In [60]:
# Initialize the model
input_size = X_train_concat_tensor_cust.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_con_custTer = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_con_custTer.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(model_con_custTer, train_loader_con_custTer, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_con_custTer, test_loader_con_custTer)

Epoch 1/10: 100%|██████████| 6250/6250 [00:31<00:00, 199.13batch/s, loss=1.039]


Epoch 1/10, Loss: 6492.232459485531


Epoch 2/10: 100%|██████████| 6250/6250 [00:32<00:00, 194.03batch/s, loss=1.022]


Epoch 2/10, Loss: 6386.859766960144


Epoch 3/10: 100%|██████████| 6250/6250 [00:33<00:00, 188.84batch/s, loss=1.018]


...

Epoch 8/10, Loss: 6315.888198435307


Epoch 9/10: 100%|██████████| 6250/6250 [00:31<00:00, 198.21batch/s, loss=1.010]


Epoch 9/10, Loss: 6312.940587043762


Epoch 10/10: 100%|██████████| 6250/6250 [00:32<00:00, 194.33batch/s, loss=1.009]


Epoch 10/10, Loss: 6307.007692813873
Accuracy: 0.50824


## First 10 Word2Vec vectors on Ternary Classification - Pretrained  Model


In [61]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_con_preTer = []
X_test_con_preTer = []

for review in ternary_word_vectors_train:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_con_preTer.append(concatenated_vector)

for review in ternary_word_vectors_test:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_con_preTer.append(concatenated_vector)



In [62]:
# Convert the concatenated features into PyTorch tensors
X_train_con_preTer_tensor = torch.tensor(X_train_con_preTer, dtype=torch.float32)
X_test_con_preTer_tensor = torch.tensor(X_test_con_preTer, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# Create DataLoader
train_dataset_con_preTer = TensorDataset(X_train_con_preTer_tensor, y_train_tensor)
test_dataset_con_preTer = TensorDataset(X_test_con_preTer_tensor, y_test_tensor)

train_loader_concat = DataLoader(train_dataset_con_preTer, batch_size=32, shuffle=True)
test_loader_concat = DataLoader(test_dataset_con_preTer, batch_size=32)



In [63]:
# Initialize the model
input_size = X_train_con_preTer_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_con_preTer = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_con_preTer.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(model_con_preTer, train_loader_concat, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_con_preTer, test_loader_concat)

Epoch 1/10: 100%|██████████| 6250/6250 [00:32<00:00, 195.26batch/s, loss=1.048]


Epoch 1/10, Loss: 6547.956055641174


Epoch 2/10: 100%|██████████| 6250/6250 [00:33<00:00, 188.12batch/s, loss=1.028]


Epoch 2/10, Loss: 6427.81138497591


Epoch 3/10: 100%|██████████| 6250/6250 [00:32<00:00, 190.85batch/s, loss=1.025]


...

Epoch 8/10, Loss: 6350.372229039669


Epoch 9/10: 100%|██████████| 6250/6250 [00:32<00:00, 193.01batch/s, loss=1.015]


Epoch 9/10, Loss: 6344.632316529751


Epoch 10/10: 100%|██████████| 6250/6250 [00:33<00:00, 187.84batch/s, loss=1.014]


Epoch 10/10, Loss: 6340.228854119778
Accuracy: 0.50066


## First 10 Word2Vec vectors on Binary Classification -  Pretrained Model


In [64]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_con_preBin = []
X_test_con_preBin = []

for review in binary_word_vectors_train:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_con_preBin.append(concatenated_vector)

for review in binary_word_vectors_test:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_con_preBin.append(concatenated_vector)



In [65]:
# Convert the concatenated features into PyTorch tensors
X_train_con_preBin_tensor = torch.tensor(X_train_con_preBin, dtype=torch.float32)
X_test_con_preBin_tensor = torch.tensor(X_test_con_preBin, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor_preBin = torch.tensor(y_train_binary, dtype=torch.long)
y_test_tensor_preBin = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoader
train_dataset_con_preBin = TensorDataset(X_train_con_preBin_tensor, y_train_tensor_preBin)
test_dataset_con_preBin = TensorDataset(X_test_con_preBin_tensor, y_test_tensor_preBin)

train_loader_con_preBin = DataLoader(train_dataset_con_preBin, batch_size=32, shuffle=True)
test_loader_con_preBin = DataLoader(test_dataset_con_preBin, batch_size=32)


In [66]:

# Initialize the model
input_size = X_train_con_preBin_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Assuming 2 classes for sentiment analysis
model_con_preBin = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_con_preBin.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(model_con_preBin, train_loader_con_preBin, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_con_preBin, test_loader_con_preBin)

Epoch 1/10: 100%|██████████| 5000/5000 [00:24<00:00, 202.31batch/s, loss=0.668]


Epoch 1/10, Loss: 3337.67026668787


Epoch 2/10: 100%|██████████| 5000/5000 [00:24<00:00, 205.42batch/s, loss=0.655]


Epoch 2/10, Loss: 3276.2301232218742


Epoch 3/10: 100%|██████████| 5000/5000 [00:25<00:00, 197.99batch/s, loss=0.651]


...

Epoch 8/10, Loss: 3197.6700633764267


Epoch 9/10: 100%|██████████| 5000/5000 [00:27<00:00, 184.44batch/s, loss=0.639]


Epoch 9/10, Loss: 3194.3690379858017


Epoch 10/10: 100%|██████████| 5000/5000 [00:24<00:00, 203.05batch/s, loss=0.638]


Epoch 10/10, Loss: 3188.5361228883266
Accuracy: 0.639275


## First 10 Word2Vec vectors on Binary Classification - Custom Word2Vec Model


In [67]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_con_custBin = []
X_test_con_custBin = []

for review in bin_cust_w2v_train:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_con_custBin.append(concatenated_vector)

for review in bin_cust_w2v_test:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_con_custBin.append(concatenated_vector)


In [68]:

# Convert the concatenated features into PyTorch tensors
X_train_con_custBin_tensor = torch.tensor(X_train_con_custBin, dtype=torch.float32)
X_test_con_custBin_tensor = torch.tensor(X_test_con_custBin, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor_custBin = torch.tensor(y_train_binary, dtype=torch.long)
y_test_tensor_custBin = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoader
train_dataset_con_custBin = TensorDataset(X_train_con_custBin_tensor, y_train_tensor_custBin)
test_dataset_con_custBin = TensorDataset(X_test_con_custBin_tensor, y_test_tensor_custBin)

train_loader_con_custBin = DataLoader(train_dataset_con_custBin, batch_size=32, shuffle=True)
test_loader_con_custBin = DataLoader(test_dataset_con_custBin, batch_size=32)



In [69]:
# Initialize the model
input_size = X_train_con_custBin_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Assuming 2 classes for sentiment analysis
model_con_custBin = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_con_custBin.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train_with_progress_bar(model_con_custBin, train_loader_con_custBin, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_con_custBin, test_loader_con_custBin)

Epoch 1/10: 100%|██████████| 5000/5000 [00:24<00:00, 205.79batch/s, loss=0.661]


Epoch 1/10, Loss: 3302.6701834201813


Epoch 2/10: 100%|██████████| 5000/5000 [00:23<00:00, 214.35batch/s, loss=0.647]


Epoch 2/10, Loss: 3237.0680083036423


Epoch 3/10: 100%|██████████| 5000/5000 [00:24<00:00, 203.30batch/s, loss=0.644]


...

Epoch 8/10, Loss: 3181.068635046482


Epoch 9/10: 100%|██████████| 5000/5000 [00:24<00:00, 205.83batch/s, loss=0.635]


Epoch 9/10, Loss: 3177.1513691544533


Epoch 10/10: 100%|██████████| 5000/5000 [00:23<00:00, 209.66batch/s, loss=0.635]


Epoch 10/10, Loss: 3173.6881232857704
Accuracy: 0.6377


### What do you conclude by comparing accuracy values you obtain with those obtained in the Simple Models section (note you can compare the accuracy values for binary classification).

1. **Simple Models (Perceptron and SVM)**:
   - Perceptron (TF-IDF): 0.81385
   - LinearSVC (TF-IDF): 0.863825
   - Perceptron (Word2Vec Pretrained Features): 0.5425
   - LinearSVC (Word2Vec Pretrained Features): 0.61595
   - Perceptron (Word2Vec Custom Features): 0.58
   - LinearSVC (Word2Vec Custom Features): 0.618325

2. **Feedforward Neural Networks (FFNN)**:
   - FFNN with Custom Word2Vec Embeddings (Average Word2Vec): 0.641675
   - FFNN with Pretrained Word2Vec Embeddings (Average Word2Vec): 0.638475
   - FFNN with Pretrained Word2Vec Embeddings (Concatenated First 10 Vectors): 0.623225
   - FFNN with Custom Word2Vec Embeddings (Concatenated First 10 Vectors): 0.63875

From the comparison, we can observe that:

- TF-IDF features consistently outperform both Word2Vec embeddings (pretrained and custom) for binary classification using the Simple Models (Perceptron and LinearSVC).This may be due to TF-IDF's ability to effectively identify the most relevant words and phrases for sentiment analysis in smaller datasets. 
- Among the Word2Vec embeddings, the FFNN with Custom Word2Vec Embeddings (Average Word2Vec) achieves the highest accuracy of 0.641675 for binary classification, slightly outperforming the FFNN with Pretrained Word2Vec Embeddings (Average Word2Vec) which has an accuracy of 0.638475.
- When considering the concatenated first 10 vectors, the FFNN with Custom Word2Vec Embeddings achieves a slightly higher accuracy of 0.63875 compared to 0.623225 obtained by the FFNN with Pretrained Word2Vec Embeddings.

Overall, TF-IDF features perform better than Word2Vec embeddings for binary sentiment analysis on this dataset. However, the FFNNs trained with custom Word2Vec embeddings show competitive performance, especially when using the average Word2Vec vectors. Additionally, the choice between pretrained and custom embeddings may depend on factors such as dataset size, domain specificity, and computational resources.

## 5. Convolutional Neural Networks 

I crafted a custom dataset class, named TextDataset, which orchestrates the loading of preprocessed text data alongside their corresponding sentiment labels. This class also harnesses a pre-trained Word2Vec model to convert the text inputs into meaningful embeddings.

My CNN architecture is designed to feature convolutional layers, followed by max-pooling layers and fully connected layers. This model ingests Word2Vec embeddings as input and outputs predictions regarding the sentiment of the text.With the architecture in place, we proceed to train the CNN model using the training data. Throughout the training process, we utilize cross-entropy loss for computing the model's loss and the Adam optimizer for updating the model's parameters.

To gauge the CNN model's efficacy, I evaluated its performance on the test data. By analyzing the accuracy metric, I ascertained the model's proficiency in accurately classifying sentiment labels.

## CNN with Binary Classification using the Word2Vec Pre-Trained Vectors

In [70]:
X_train, X_test, y_train, y_test = train_test_split(balanced_data['Review'], balanced_data['sentiment_class'], test_size=0.2, random_state=42)
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(binary_data['Review'], binary_data['binary_sentiment'], test_size=0.2, random_state=42)


In [71]:
# Preprocess text data
def preprocess_text(text, max_length=50):
    tokens = text.split()[:max_length]  # Limit maximum review length
    padded_tokens = tokens + ['<PAD>'] * (max_length - len(tokens))  # Pad shorter reviews
    return padded_tokens


In [72]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, wv, max_length=50):
        self.texts = texts
        self.labels = labels
        self.wv = wv
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        tokens = preprocess_text(text, self.max_length)
        vector = np.zeros((self.max_length, wv.vector_size), dtype=np.float32)  # Keep as float32
        for i, token in enumerate(tokens):
            if token in self.wv:
                vector[i] = self.wv[token]
            else:
                vector[i] = np.zeros(wv.vector_size)
        return torch.tensor(vector, dtype=torch.float32), torch.tensor(label, dtype=torch.long)  # Ensure dtype is float32


In [73]:

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(input_size, 50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(50, 10, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(10 * 12, output_size)  # Adjust the input size for the linear layer

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 10 * 12)  # Adjust the reshaping operation
        x = self.fc(x)
        return nn.functional.softmax(x, dim=1)


def train_cnn(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, (inputs, labels) in progress_bar:
            optimizer.zero_grad()
            outputs = model(inputs.permute(0, 2, 1))  # Permute input dimensions for Conv1D
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix({'loss': running_loss / (i + 1)})
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


# Evaluation
def evaluate_cnn(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.permute(0, 2, 1))  # Permute input dimensions for Conv1D
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct / total}")

In [74]:
# Filter and prepare data as described initially
# Assuming X_train_binary, y_train_binary, X_test_binary, y_test_binary are already defined

train_dataset = TextDataset(X_train_binary, y_train_binary, wv)
test_dataset = TextDataset(X_test_binary, y_test_binary, wv)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)




In [75]:
# Initialize the model
input_size = wv.vector_size
output_size = 2  # Binary classification
model = CNN(input_size, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_cnn(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_cnn(model, test_loader)

Epoch 1/10: 100%|██████████| 5000/5000 [01:53<00:00, 44.12it/s, loss=0.479]


Epoch 1/10, Loss: 0.4790187286913395


Epoch 2/10: 100%|██████████| 5000/5000 [01:38<00:00, 50.72it/s, loss=0.448]


Epoch 2/10, Loss: 0.4483756018579006


Epoch 3/10: 100%|██████████| 5000/5000 [01:39<00:00, 50.22it/s, loss=0.436]


...

Epoch 8/10, Loss: 0.4090325618505478


Epoch 9/10: 100%|██████████| 5000/5000 [01:39<00:00, 50.33it/s, loss=0.406]


Epoch 9/10, Loss: 0.4060825840473175


Epoch 10/10: 100%|██████████| 5000/5000 [01:40<00:00, 49.65it/s, loss=0.403]


Epoch 10/10, Loss: 0.4034036285281181
Accuracy: 0.8642


## CNN with Ternary Classification using the Word2Vec Pre-Trained Vectors

In [76]:
# Filter and prepare data as described initially
# Assuming X_train_binary, y_train_binary, X_test_binary, y_test_binary are already defined

train_dataset = TextDataset(X_train, y_train, wv)
test_dataset = TextDataset(X_test, y_test, wv)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)




In [77]:
# Initialize the model
input_size = wv.vector_size
output_size = 3  # Binary classification
model = CNN(input_size, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_cnn(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_cnn(model, test_loader)

Epoch 1/10: 100%|██████████| 6250/6250 [02:07<00:00, 49.04it/s, loss=0.875]


Epoch 1/10, Loss: 0.8754354429531097


Epoch 2/10: 100%|██████████| 6250/6250 [02:03<00:00, 50.57it/s, loss=0.84] 


Epoch 2/10, Loss: 0.8401069899082184


Epoch 3/10: 100%|██████████| 6250/6250 [02:03<00:00, 50.62it/s, loss=0.828]


...

Epoch 8/10, Loss: 0.7947168046283722


Epoch 9/10: 100%|██████████| 6250/6250 [02:04<00:00, 50.30it/s, loss=0.791]


Epoch 9/10, Loss: 0.7905362372493744


Epoch 10/10: 100%|██████████| 6250/6250 [02:05<00:00, 49.77it/s, loss=0.787]


Epoch 10/10, Loss: 0.7866367936229706
Accuracy: 0.7042


## CNN with Binary Classification using the Word2Vec Custom Vectors

In [78]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class TextDataset(Dataset):
    def __init__(self, texts, labels, wv_model, max_length=50):
        """
        texts: Series or list of text documents
        labels: Series or list of labels
        wv_model: The trained Word2Vec model (Gensim)
        max_length: Maximum length of tokens in a document
        """
        self.texts = texts
        self.labels = labels
        self.wv_model = wv_model
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        vector = np.zeros((self.max_length, self.wv_model.vector_size))

        # Convert text to vector
        for i, word in enumerate(text[:self.max_length]):
            if word in self.wv_model.wv:
                vector[i] = self.wv_model.wv[word]
            else:
                vector[i] = np.zeros(self.wv_model.vector_size)

        return torch.tensor(vector, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [79]:
# Assuming balanced_data['Review'] and the corresponding labels are pandas Series
X_train_binary = X_train_binary.reset_index(drop=True)
y_train_binary = y_train_binary.reset_index(drop=True)
X_test_binary = X_test_binary.reset_index(drop=True)
y_test_binary = y_test_binary.reset_index(drop=True)

# Now, pass these to your TextDataset instances
train_dataset = TextDataset(X_train_binary, y_train_binary, model_cust)
test_dataset = TextDataset(X_test_binary, y_test_binary, model_cust)


In [80]:
# Filter and prepare data as described initially
# Assuming X_train_binary, y_train_binary, X_test_binary, y_test_binary are already defined



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)




In [81]:
# Initialize the model
input_size = model_cust.vector_size
output_size = 2  # Binary classification
model = CNN(input_size, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_cnn(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_cnn(model, test_loader)

Epoch 1/10: 100%|██████████| 5000/5000 [01:35<00:00, 52.57it/s, loss=0.594]


Epoch 1/10, Loss: 0.5939369991242885


Epoch 2/10: 100%|██████████| 5000/5000 [01:35<00:00, 52.56it/s, loss=0.564]


Epoch 2/10, Loss: 0.564359641867876


Epoch 3/10: 100%|██████████| 5000/5000 [01:35<00:00, 52.48it/s, loss=0.555]


...

Epoch 8/10, Loss: 0.5398669212222099


Epoch 9/10: 100%|██████████| 5000/5000 [01:36<00:00, 51.68it/s, loss=0.539]


Epoch 9/10, Loss: 0.5388882895052433


Epoch 10/10: 100%|██████████| 5000/5000 [01:37<00:00, 51.47it/s, loss=0.537]


Epoch 10/10, Loss: 0.5372337187767029
Accuracy: 0.75025


## CNN with Ternary Classification using the Word2Vec Custom Vectors

In [82]:
# Assuming balanced_data['Review'] and the corresponding labels are pandas Series
X_train = X_train.reset_index(drop=True)
y_train= y_train.reset_index(drop=True)
X_test= X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Now, pass these to your TextDataset instances
train_dataset = TextDataset(X_train, y_train, model_cust)
test_dataset = TextDataset(X_test, y_test, model_cust)


In [83]:
# Filter and prepare data as described initially
# Assuming X_train_binary, y_train_binary, X_test_binary, y_test_binary are already defined



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)




In [84]:
# Initialize the model
input_size = model_cust.vector_size
output_size = 3  # Binary classification
model = CNN(input_size, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_cnn(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_cnn(model, test_loader)

Epoch 1/10: 100%|██████████| 6250/6250 [02:04<00:00, 50.09it/s, loss=0.975]


Epoch 1/10, Loss: 0.975375921163559


Epoch 2/10: 100%|██████████| 6250/6250 [02:02<00:00, 51.01it/s, loss=0.949]


Epoch 2/10, Loss: 0.9491483219814301


Epoch 3/10: 100%|██████████| 6250/6250 [02:03<00:00, 50.56it/s, loss=0.941]


...

Epoch 8/10, Loss: 0.9300689810371399


Epoch 9/10: 100%|██████████| 6250/6250 [02:12<00:00, 47.04it/s, loss=0.929]


Epoch 9/10, Loss: 0.9285816311931611


Epoch 10/10: 100%|██████████| 6250/6250 [02:12<00:00, 47.00it/s, loss=0.928]


Epoch 10/10, Loss: 0.9278761985492706
Accuracy: 0.6076
